#### THIS is a capstone project from--Microsoft Professional Program in Artificial Intelligence | edX course (DAT264x)
#### natural language processing, multi-class, multi-label classification challenge

In [1]:
#call modules
from sklearn.svm import LinearSVC, SVC
from sklearn.decomposition import PCA,TruncatedSVD
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.linear_model import RidgeClassifier,PassiveAggressiveClassifier,SGDClassifier,LogisticRegression
from sklearn.naive_bayes import BernoulliNB, ComplementNB, MultinomialNB

In [3]:
#call modules
from sklearn.metrics import classification_report, accuracy_score, recall_score,precision_score, confusion_matrix,jaccard_similarity_score,roc_auc_score,f1_score,hamming_loss,make_scorer
import nltk
from sklearn.preprocessing import MultiLabelBinarizer
import re
from sklearn.multioutput import ClassifierChain
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV,train_test_split
from sklearn import naive_bayes ,metrics
from sklearn.feature_selection import SelectPercentile, f_classif,SelectKBest, chi2
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier


In [4]:
#call modules
import pandas as pd
import numpy as np
import nlp_functions as nlp
from scipy import stats


In [5]:
#load origin data from csv file
train_values=pd.read_csv('/home/ajshemi/DAT264x_Identifying_Topics_of_World_Bank_Publications/train_values.csv',index_col=0,header=0)
test_values=pd.read_csv('/home/ajshemi/DAT264x_Identifying_Topics_of_World_Bank_Publications/test_values.csv',index_col=0,header=0)
train_labels=pd.read_csv('/home/ajshemi/DAT264x_Identifying_Topics_of_World_Bank_Publications/train_labels.csv',index_col=0,header=0)

In [7]:
#clean up train data and save data (not shown here)
#retrieve data, load data
X=pd.read_csv('X_data_1.csv',header=0,dtype=np.object_).iloc[:,-1]

In [10]:
#handling np.nan
X_nan=pd.Series(np.where(X.isna()==True,X.values.astype('U'),X.values))

In [11]:
X=X_nan
#X_Set=X_Set_nan

In [12]:
vec = TfidfVectorizer(max_features=20000,stop_words='english',ngram_range=(1,2))

In [13]:
#split training data into training dataset and testing dataset
X_train_raw, X_test_raw, y_train, y_test = train_test_split(X.values, train_labels.values,test_size=0.25,random_state=0)
#X_train_raw, X_test_raw, y_train, y_test = train_test_split(pd.read_csv('X_data_1.csv',header=0,dtype= np.object_).iloc[:,-1].values, train_labels.values,test_size=0.0,random_state=0)

In [14]:
#fit_transform training data
#X_train=vectorizer.fit_transform(X_train_raw).toarray()
X_train=vec.fit_transform(X_train_raw)#toarray() #?list object has no attribute lower #using join removes the problem

In [15]:
X_train.shape #, X_test.shape

(14122, 20000)

In [16]:
X_test=vec.transform(X_test_raw)#.toarray()

In [17]:
df=pd.DataFrame()
#df_pred=pd.DataFrame()
col=[]
for i in range(y_train.shape[1]):
    #select best features
    select=SelectKBest(chi2, k=7500)
    #select=SelectKBest(f_classif, k=7500)
    #select=SelectPercentile(f_classif,percentile=10)
    
    #fit and transform data using best features
    X_train_sel=select.fit_transform(X_train,y_train[:,i]).toarray()
    X_test_sel=select.transform(X_test).toarray()
    
    #called model and fit model
    base_SVC=LinearSVC(tol=1e-3,max_iter=1000,class_weight=None)
    ovr_SVC=OneVsRestClassifier(base_SVC)
    ovr_SVC.fit(X_train_sel, y_train[:,i])
    
    #predict model
    col = 'y_pred_%d' % i
    df[col] = ovr_SVC.predict(X_test_sel)
    
    #X_Test_sel=select.transform(X_Test).toarray()
    #ovr_RC.predict(X_test_sel)
    #df_pred[col]=ovr_SVC.predict(X_Test_sel)    

In [ ]:
metrics.f1_score(y_test, df.values,average='micro'),metrics.accuracy_score(y_test, df.values)

In [ ]:
#Table of selected results
(0.6196887974855458, 0.22897196261682243)(1,2)SelectKBest(chi2, k=7500),LinearSVC(tol=1e-3,max_iter=1000,class_weight=None)
(0.6150645347717202, 0.22323704333050126)(1,1)SelectKBest(chi2, k=10000),LinearSVC(tol=1e-3,max_iter=1000,class_weight=None)
(0.6070543824466202, 0.22323704333050126)(2,2)SelectKBest(chi2, k=10000),LinearSVC(tol=1e-3,max_iter=1000,class_weight=None)
(0.6246828143021915, 0.2336448598130841)(1,2)SelectKBest(chi2, k=10000),LinearSVC(tol=1e-3,max_iter=1000,class_weight=None)
(0.6157903022670025, 0.225785896346644)(1,3)SelectKBest(chi2, k=5000),LinearSVC(tol=1e-3,max_iter=1000,class_weight=None)
(0.5903346925592474, 0.2143160577740017) (2,3)SelectKBest(chi2, k=5000),LinearSVC(tol=1e-3,max_iter=1000,class_weight=None)
(0.6247308520455245, 0.23237043330501275)(1,2)SelectKBest(chi2, k=10000),LinearSVC(tol=1e-3,max_iter=1000,class_weight=None)
(0.6178346441288101, 0.22536108751062023)(1,2)SelectKBest(chi2, k=5000),LinearSVC(tol=1e-3,max_iter=1000,class_weight=None)
(0.5809917355371901, 0.12213254035683942) selectPercentile=10 LinearSVC(tol=1e-3,max_iter=1000,class_weight='balanced')
(0.5846345385692794, 0.12383177570093458) SelectKBest(chi2, k=2028) LinearSVC(tol=1e-3,max_iter=1000,class_weight='balanced')
(0.6033900996387599, 0.21771452846219203) SelectKBest(chi2, k=2028) LinearSVC(tol=1e-3,max_iter=1000,class_weight=None)
(0.6050980392156863, 0.21771452846219203) SelectKBest(chi2, k=2028) SGDClassifier(loss='modified_huber',tol=1e-3)
(0.5617931148353009, 0.19923534409515717) SelectKBest(chi2, k=2028) GradientBoostingClassifier()

#### principal component analysis approach

In [ ]:
alt_pca=TruncatedSVD(n_components=5000)
alt_pca.fit(X_train)
X_train_alt_pca=alt_pca.transform(X_train)
X_test_alt_pca=alt_pca.transform(X_test)
X_test_alt_pca.shape,X_train_alt_pca.shape,type(X_test_alt_pca),type(X_train_alt_pca)

In [ ]:
#pca=PCA(n_components=2500)
#pca.fit(X_train)
#X_train_pca=pca.transform(X_train)
#X_test_pca=pca.transform(X_test)

In [ ]:
#X_Test_alt_pca=alt_pca.transform(X_Test)

In [ ]:
#hyper parameter testing. RidgeClassifier. alternate pca
f1score=[]
accuscore=[]
#max_iterations=[1000,5000]
alpha=[0.001,0.0001]
tol=[1e-3,1e-6]
#class_weights=[None,'balanced']
for i in alpha:
    for j in tol:
        #base_SVC=LinearSVC(tol=1e-3,max_iter=i,class_weight=j)
        #ovr_SVC=OneVsRestClassifier(base_SVC)
        #ovr_SVC.fit(X_train_alt_pca, y_train)
        
        base_RC=RidgeClassifier(tol=j,class_weight=None,alpha=i)
        ovr_RC=OneVsRestClassifier(base_RC)
        ovr_RC.fit(X_train_alt_pca, y_train)
        f1score.append(metrics.f1_score(y_test, ovr_RC.predict(X_test_alt_pca),average='micro'))
        accuscore.append(metrics.accuracy_score(y_test, ovr_RC.predict(X_test_alt_pca)))

print(f1score)
print(accuscore)

In [ ]:
tol=1e-3
0.6182317701377494, 0.22026338147833474)

In [ ]:
#metrics.f1_score(y_test, ovr_RC.predict(X_test),average='micro'),metrics.accuracy_score(y_test, ovr_RC.predict(X_test))
metrics.f1_score(y_test, ovr_RC.predict(X_test_alt_pca),average='micro'),metrics.accuracy_score(y_test, ovr_RC.predict(X_test_alt_pca))

In [ ]:
#jaccard_similarity_score(y_test, ovr_RC.predict(X_test)),hamming_loss(y_test, ovr_RC.predict(X_test))
jaccard_similarity_score(y_test, ovr_RC.predict(X_test_alt_pca)),hamming_loss(y_test, ovr_RC.predict(X_test_alt_pca))

#### Clean up and prepare test data

In [ ]:
#given test data; test_values
X_Test=test_values['doc_text']

In [ ]:
#data preparation; clean test data
X_Test=X_Test.apply(lambda x: nlp.clean_up_text(x))

In [ ]:
#use split_words function
X_Test=X_Test.apply(lambda x: split_words(str(x)))

In [ ]:
#use splitW function
X_Test=X_Test.apply(lambda x: splitW(x))

In [ ]:
#tokenize test data
X_Test=X_Test.apply(lambda x: nlp.tokenize(x, min_char=4))

In [ ]:
#lemmatize test data
X_Test=X_Test.apply(lambda x: [lm.lemmatize(word) for word in x])

In [ ]:
#create string of tokenized text
X_Test=X_Test.apply(lambda x: ' '.join(x))

In [ ]:
#stem text
X_Test=X_Test.apply(lambda x: [ps.stem(word) for word in x])

In [ ]:
#save data to csv file
pd.DataFrame(X_Test).to_csv('X_Test_data_1.csv',encoding='utf-8',index=False)

In [ ]:
#retrieve data, load data
X_Test=pd.read_csv('X_Test_data_1.csv',header=0,dtype= np.object_).iloc[:,-1]#.values

In [ ]:
#handling np.nan
X_Test_nan=pd.Series(np.where(X_Test.isna()==True,X_Test.values.astype('U'),X_Test.values))
X_Test=X_Test_nan

In [ ]:
#transform test data
#X_Test=vectorizer.transform(X_Test).toarray()
X_Test=vectorizer.transform(pd.read_csv('X_Test_data_1.csv',header=0,dtype= np.object_ ).iloc[:,-1].values)#.toarray()